In [140]:
import pickle as pkl
import cv2
import ipywidgets as widgets
from IPython.display import display
import os
import re
from collections import defaultdict
from pprint import pprint
import torch
import numpy as np

In [166]:
video_id = "00T1E.mp4"

In [167]:
pred = pkl.load(open(f'output/sgdet/{video_id}/pred.pkl', 'rb'))

In [168]:
pred.keys()

dict_keys(['boxes', 'scores', 'distribution', 'pred_labels', 'features', 'fmaps', 'im_info', 'indices', 'object_features', 'object_mem_features', 'pred_scores', 'pair_idx', 'im_idx', 'human_idx', 'union_feat', 'union_box', 'spatial_masks', 'rel_features', 'rel_mem_features', 'attention_distribution', 'spatial_distribution', 'contacting_distribution'])

In [169]:
for k in pred.keys():
    print(f"{k} -- {len(pred[k]) if type(pred[k]) == list else pred[k].shape}")


boxes -- torch.Size([598, 5])
scores -- torch.Size([598])
distribution -- torch.Size([598, 36])
pred_labels -- torch.Size([598])
features -- torch.Size([598, 2048])
fmaps -- torch.Size([29, 1024, 67, 38])
im_info -- torch.Size([])
indices -- 2
object_features -- torch.Size([598, 2376])
object_mem_features -- torch.Size([598, 2376])
pred_scores -- torch.Size([598])
pair_idx -- torch.Size([569, 2])
im_idx -- torch.Size([569])
human_idx -- torch.Size([29, 1])
union_feat -- torch.Size([569, 1024, 7, 7])
union_box -- torch.Size([569, 5])
spatial_masks -- torch.Size([569, 2, 27, 27])
rel_features -- torch.Size([569, 1936])
rel_mem_features -- torch.Size([569, 1936])
attention_distribution -- torch.Size([569, 3])
spatial_distribution -- torch.Size([569, 6])
contacting_distribution -- torch.Size([569, 17])


In [170]:
boxes = pred["boxes"]
scores = pred["scores"]
distribution = pred["distribution"]
pred_labels = pred["pred_labels"]
features = pred["features"]
fmaps = pred["fmaps"]
im_info = pred["im_info"]
indices = pred["indices"]
object_features = pred["object_features"]
object_mem_features = pred["object_mem_features"]
pred_scores = pred["pred_scores"]
pair_idx = pred["pair_idx"]
im_idx = pred["im_idx"]
human_idx = pred["human_idx"]
union_feat = pred["union_feat"]
union_box = pred["union_box"]
spatial_masks = pred["spatial_masks"]
rel_features = pred["rel_features"]
rel_mem_features = pred["rel_mem_features"]
attention_distribution = pred["attention_distribution"]
spatial_distribution = pred["spatial_distribution"]
contacting_distribution = pred["contacting_distribution"]

In [171]:
contacting_distribution[:10]

tensor([[2.8190e-08, 2.0034e-07, 1.5190e-05, 3.8861e-05, 1.3654e-08, 4.2722e-07,
         1.0221e-05, 1.4609e-02, 1.2848e-04, 1.0712e-01, 7.9500e-03, 1.0000e+00,
         2.5458e-06, 1.3349e-06, 3.8643e-05, 1.1045e-06, 3.1476e-06],
        [7.6223e-08, 1.1537e-06, 8.6423e-06, 2.9885e-06, 2.7553e-08, 2.0293e-07,
         8.1284e-05, 1.3019e-02, 1.6941e-04, 3.3081e-01, 1.9713e-02, 1.0000e+00,
         3.1059e-06, 1.1244e-06, 5.2268e-05, 4.5091e-07, 3.5214e-06],
        [5.8940e-08, 3.4347e-07, 4.2568e-05, 8.0987e-06, 1.2624e-08, 3.0400e-07,
         6.4397e-06, 8.3831e-03, 2.8405e-04, 2.3721e-01, 4.7566e-04, 1.0000e+00,
         5.0917e-06, 4.3341e-06, 2.1940e-04, 3.0949e-06, 7.6478e-06],
        [3.3849e-07, 1.4925e-05, 7.1869e-05, 6.8641e-06, 2.8320e-08, 2.0416e-06,
         2.4635e-05, 1.5699e-02, 2.9744e-05, 6.0464e-01, 3.1953e-03, 1.0000e+00,
         8.2093e-05, 1.5342e-05, 6.6034e-04, 6.8736e-06, 1.1548e-04],
        [1.4930e-07, 1.2458e-05, 1.7582e-05, 7.1264e-06, 2.2270e-08, 8.7

In [172]:
# Load txt file
object_classes = []
with open('/home/pali/datasets/charades/dataset/ag/annotations/object_classes.txt', 'r') as file:
    for line in file:
        object_classes.append(line.strip()) 
object_classes

['person',
 'bag',
 'bed',
 'blanket',
 'book',
 'box',
 'broom',
 'chair',
 'closetcabinet',
 'clothes',
 'cupglassbottle',
 'dish',
 'door',
 'doorknob',
 'doorway',
 'floor',
 'food',
 'groceries',
 'laptop',
 'light',
 'medicine',
 'mirror',
 'papernotebook',
 'phonecamera',
 'picture',
 'pillow',
 'refrigerator',
 'sandwich',
 'shelf',
 'shoe',
 'sofacouch',
 'table',
 'television',
 'towel',
 'vacuum',
 'window']

In [173]:
len(object_classes  )

36

In [174]:
relationship_classes = []
with open('/home/pali/datasets/charades/dataset/ag/annotations/relationship_classes.txt', 'r') as file:
    for line in file:
        relationship_classes.append(line.strip()) 
len(relationship_classes), relationship_classes

(26,
 ['lookingat',
  'notlookingat',
  'unsure',
  'above',
  'beneath',
  'infrontof',
  'behind',
  'onthesideof',
  'in',
  'carrying',
  'coveredby',
  'drinkingfrom',
  'eating',
  'haveitontheback',
  'holding',
  'leaningon',
  'lyingon',
  'notcontacting',
  'otherrelationship',
  'sittingon',
  'standingon',
  'touching',
  'twisting',
  'wearing',
  'wiping',
  'writingon'])

In [175]:
# Path to the directory containing the frames
frames_dir = f'/home/pali/datasets/charades/dataset/ag/frames/{video_id}/'

# Load and sort the frames
frame_files = os.listdir(frames_dir)
frame_files = sorted(frame_files, key=lambda x: int(re.search(r'\d+', x).group()))

frames = []
for frame_file in frame_files:
    frame_path = os.path.join(frames_dir, frame_file)
    frame = cv2.imread(frame_path)
    if frame is not None:
        frames.append(frame)

print(f"Total frames: {len(frames)}")


Total frames: 36


In [176]:
def interactive_plot(data, frames, threshold=0.7):
    # Function to display the frame with bounding boxes
    def display_frame(frame_index):
        frame = frames[frame_index].copy()
        
        # # Draw bounding boxes for the current frame
        # for bbox in frame_bboxes[frame_index]:
        #     x1, y1, x2, y2, label, score = bbox
        #     cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            
        #     text = f'{label} {score:.2f}'

        #     # Calculate the size of the label to create a background for readability
        #     (label_width, label_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        #     cv2.rectangle(frame, (int(x1), int(y1) - label_height - baseline), (int(x1) + label_width, int(y1)), (0, 255, 0), cv2.FILLED)
        #     cv2.putText(frame, text, (int(x1), int(y1) - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
        
        for sgg in data[frame_index]:

            # subject = sgg["subject"]
            # object = sgg["object"]
            x1, y1, x2, y2, = sgg["box"]
            score = sgg["score"]
            pred_label = sgg["pred_label"]
            # pred_score = sgg["pred_score"]
            # attn = sgg["attn"]
            # spat = sgg["spat"]
            # cont = sgg["cont"]



            if score > threshold:
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                # pprint.pprint(sgg)
                # print(f'<{subject}, {object}>')
                # max_attn = torch.argmax(attn)
                # max_spat = torch.argmax(spat)
                # max_cont = torch.argmax(cont)
                # print(f'Attn: {relationship_classes[max_attn]} - score: {attn[max_attn]}')
                # print(f'Spat: {relationship_classes[max_spat]} - score: {spat[max_spat]}')
                # print(f'Cont: {relationship_classes[max_cont]} - score: {cont[max_cont]}')
                pprint({
                    "pred_label": object_classes[pred_label-1],
                    "score": score
                })
                print('*'*60)
            # print(f'<{subject}, {object}>')

        # Display the frame with bounding boxes
        frame_widget = widgets.Image(value=cv2.imencode('.jpg', frame)[1].tobytes())
        display(frame_widget)

    # Create the interactive widgets
    frame_index_slider = widgets.IntSlider(min=0, max=len(data)-1, value=0)

    # Update the frame when the slider value changes
    return widgets.interactive(display_frame, frame_index=frame_index_slider)

In [177]:
pred_labels

tensor([16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  1,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16,  1, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16,  1, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  1, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16,  1, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16,  1, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  1, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16,  1, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,  1, 16, 16, 16, 16, 16,
        16, 16, 16, 16, 16, 16, 16, 16, 

In [178]:
data = defaultdict(list)
# Add edges (relationships)
for i, box in enumerate(boxes):
    # img_idx = int(im_idx[i])
    # subj_idx, obj_idx = pair
    # box = boxes[i]
    # score = scores[i]
    # pred_score = pred_scores[i]
    # attn = attention_distribution[i]
    # spat = spatial_distribution[i]
    # cont = contacting_distribution[i]
    # pred_label = pred_labels[i]

    # # print(f'img_idx:{img_idx} - <{object_classes[pred_labels[subj_idx]-1]}, {object_classes[pred_labels[obj_idx]-1]}>')
    # data[im_i].append({
    #     'subject': object_classes[pred_labels[subj_idx]-1],
    #     'object': object_classes[pred_labels[obj_idx]-1],
    #     'box': box[1:],
    #     'score': score,
    #     'pred_score': pred_score,
    #     'attn': attn,
    #     'spat': spat,
    #     'cont': cont
    # })
    im_i = int(box[0])
    bbox = box[1:]
    score = scores[i]
    pred_label = pred_labels[i]
    dist = distribution[i]

    print(torch.argmax(dist))
    
    data[im_i].append({
        'box': bbox,
        'score': score,
        'pred_label': pred_label
    })


tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)
tensor(15)

In [179]:
for i, p in enumerate(pred_labels):
    if pred_scores[i]> 0.85:
        print(f'{i} - {object_classes[p-1]} - {pred_scores[i]}')

0 - floor - 0.8885332942008972
1 - floor - 0.8755560517311096
2 - floor - 0.8661300539970398
3 - floor - 0.8520028591156006
4 - floor - 0.851778507232666
5 - floor - 0.8515697717666626
21 - floor - 0.897333562374115
22 - floor - 0.8957229852676392
23 - floor - 0.8867518901824951
24 - floor - 0.8530063033103943
25 - floor - 0.8525620698928833
43 - floor - 0.9227115511894226
44 - floor - 0.9180519580841064
45 - floor - 0.9145464897155762
46 - floor - 0.9091394543647766
47 - floor - 0.8959729075431824
48 - floor - 0.8957905173301697
49 - floor - 0.8932345509529114
50 - floor - 0.8890399932861328
51 - floor - 0.8821278214454651
52 - floor - 0.8802666068077087
53 - floor - 0.8634189367294312
54 - floor - 0.8568608164787292
55 - floor - 0.8547019362449646
67 - floor - 0.9232240915298462
68 - floor - 0.922713577747345
69 - floor - 0.9168580770492554
70 - floor - 0.9164089560508728
71 - floor - 0.8974531888961792
72 - floor - 0.8809390068054199
73 - floor - 0.8778529167175293
74 - floor - 0.87

In [180]:
interactive_plot(data, frames, threshold=0.967)

interactive(children=(IntSlider(value=0, description='frame_index', max=28), Output()), _dom_classes=('widget-…